In [2]:
url = "https://learn.microsoft.com/en-us/intune/intune-service/fundamentals/what-is-intune"

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")


In [4]:
##data ingestion

from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader= WebBaseLoader(url)

In [6]:
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://learn.microsoft.com/en-us/intune/intune-service/fundamentals/what-is-intune', 'title': 'What is Microsoft Intune | Microsoft Learn', 'description': 'Microsoft Intune manages users and devices, simplifies app management and automated policy deployment, and integrates with mobile threat defense. It connects to Managed Google Play, Apple tokens and certificates, and Teamviewer for remote assistance. Can use MDM or MAM to protect data, configure devices, and simplify access to company resources.', 'language': 'en-us'}, page_content=" \n\n\nWhat is Microsoft Intune | Microsoft Learn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\tSkip to main content\n\t\t\n\n\t\t\tSkip to Ask Learn chat experience\n\t\t\n\n\n\n\nThis browser is no longer supported.\n\n\t\t\t\t\t\tUpgrade to Microsoft Edge to take advantage of the latest f

In [8]:
##splitter-- text to chunk
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents =splitter.split_documents(docs)

In [10]:
documents

[Document(metadata={'source': 'https://learn.microsoft.com/en-us/intune/intune-service/fundamentals/what-is-intune', 'title': 'What is Microsoft Intune | Microsoft Learn', 'description': 'Microsoft Intune manages users and devices, simplifies app management and automated policy deployment, and integrates with mobile threat defense. It connects to Managed Google Play, Apple tokens and certificates, and Teamviewer for remote assistance. Can use MDM or MAM to protect data, configure devices, and simplify access to company resources.', 'language': 'en-us'}, page_content='What is Microsoft Intune | Microsoft Learn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\tSkip to main content\n\t\t\n\n\t\t\tSkip to Ask Learn chat experience\n\t\t\n\n\n\n\nThis browser is no longer supported.\n\n\t\t\t\t\t\tUpgrade to Microsoft Edge to take advantage of the latest features

In [11]:
##vector
#use embedding tech>>vector db

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embedding=embeddings)

In [12]:
vectorstoredb

In [13]:
#query from vectorstore DB
query= "What is self-service features?"
result= vectorstoredb.similarity_search(query)
result[0].page_content

'For more information, go to Manage apps using Microsoft Intune.\n✅ Automate policy deployment\nYou can create policies for apps, security, device configuration, compliance, Conditional Access, and more. When the policies are ready, you can deploy these policies to your user groups and device groups. To receive these policies, the devices only need internet access.\nFor more information, go to Assign policies in Microsoft Intune.\n✅ Use the self-service features\nEmployees and students can use the Company Portal app and website to reset a PIN/password, install apps, join groups, and more. You can customize the Company Portal to help reduce support calls.\nFor more information, go to Configure the Intune Company Portal apps, Company Portal website, and Intune app.\n✅ Integrate with mobile threat defense'

In [14]:
#retrieval chain, document chain
from langchain.chains.combine_documents  import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(model="gpt-4o")


In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    ("""
    
    Answer the following question based on following context:
    <context>
    {context}
    </context>
    
    
    """)
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n\n    Answer the following question based on following context:\n    <context>\n    {context}\n    </context>\n\n\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001E944114C50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E944115A10>, root_client=<openai.OpenAI object at 0x000001E944114690>, root_async_client=<openai.AsyncOpenAI object at 0x000001E944115690>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kw

In [17]:

from langchain_core.documents import Document


document_chain.invoke(
    {
        "input": "Intune policies are applied to how many device platform?",
        "context" : [Document(page_content="Intune integrates with Microsoft Defender for Endpoint and third party partner services. With these services, the focus is on endpoint security. You can create policies that respond to threats, do real-time risk analysis, and automate remediation.")]
    }
)


'What does Intune focus on when integrating with Microsoft Defender for Endpoint and third-party partner services?'

In [18]:
##retriever---can be consider as an interface and responsible for way getting a data from vectorstore DB--pathway

retriever= vectorstoredb.as_retriever()

In [19]:
from langchain.chains import create_retrieval_chain
retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [20]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E917F88410>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n\n    Answer the following question based on following context:\n    <context>\n    {context}\n    </context>\n\n\n    '), additional_kwargs={})])


In [21]:
#response

response = retriever_chain.invoke({"input": "Intune policies are applied to how many device platform?"})
response['answer']

"Based on the provided context, here are the key points regarding Microsoft Intune's capabilities in device and application management:\n\n1. **Device Management (MDM)**: \n   - Organization-owned devices are enrolled in Intune for Mobile Device Management (MDM), which is device-centric. \n   - Intune enables configuration of device features based on user roles, such as allowing Wi-Fi access only for organization accounts.\n   - Admin teams can manage devices fully, including user identities, app installations, and data access.\n   - Policies can be deployed to devices during the enrollment process, making them ready for use upon completion.\n\n2. **Application Management (MAM)**:\n   - For personal devices under Bring Your Own Device (BYOD) scenarios, Intune offers Mobile Application Management (MAM), which is user-centric.\n   - MAM focuses on protecting app data, ensuring secure access and data protection within apps regardless of the device.\n\n3. **Policy Deployment and Automation

In [22]:
response

{'input': 'Intune policies are applied to how many device platform?',
 'context': [Document(id='4377970f-29d1-4dc8-8446-669c4d4ab104', metadata={'source': 'https://learn.microsoft.com/en-us/intune/intune-service/fundamentals/what-is-intune', 'title': 'What is Microsoft Intune | Microsoft Learn', 'description': 'Microsoft Intune manages users and devices, simplifies app management and automated policy deployment, and integrates with mobile threat defense. It connects to Managed Google Play, Apple tokens and certificates, and Teamviewer for remote assistance. Can use MDM or MAM to protect data, configure devices, and simplify access to company resources.', 'language': 'en-us'}, page_content="Enroll in device management, application management, or both\n✅ Organization-owned devices are enrolled in Intune for mobile device management (MDM). MDM is device centric, so device features are configured based on who needs them. For example, you can configure a device to allow access to Wi-Fi, but

In [ ]:
##using ollama techniques doing same 

